In [1]:
# Parameters
DEVICE_NUM=4
MODEL_SIZE=70
DATASET="fakenewsnet"
VERBOSE=False

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = str(DEVICE_NUM)
CACHE_PATH = f"../data/caches/{MODEL_SIZE}/cache.jsonl"
DATASET_PATH = f"../data/{DATASET}.csv"
SIGNALS_PATH = "../data/signals.csv"

assert os.path.exists(DATASET_PATH)
assert os.path.exists(SIGNALS_PATH)
if not os.path.exists(CACHE_PATH):
    os.makedirs(CACHE_PATH)

In [2]:
import pandas as pd
from utils import llama_chat_hf
import torch
import json
from tqdm import tqdm
from sklearn.metrics import f1_score, accuracy_score, classification_report
import hashlib

/home/jaugusto97/miniconda3/envs/cred_signalsv2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print("Device Name:", torch.cuda.get_device_name(), "Device Number:", DEVICE_NUM)

'NVIDIA A100-PCIE-40GB'

In [23]:
df = pd.read_csv(DATASET_PATH)
signal_df = pd.read_csv(SIGNALS_PATH)

,url,text,keywords,authors,canonical_link,title,publish_date,source,summary,id,objective,dataset,article_md5
7914,https://www.newidea.com.au/prince-harrys-exerc...,"Her soon-to-be husband, Prince Harry, has been...",[],['Nadia S'],https://www.newidea.com.au/prince-harrys-exerc...,Prince Harry's exercise regime ahead of weddin...,1.522115e+09,https://www.newidea.com.au,NaN,923222,real,gossipcop,62fd85373cd9dcda599a09e9a41dec00
18159,www.refinery29.com/2017/05/155114/selena-gomez...,Unbothered\n\nThe Artists Who Made Us Fall In ...,[],"['Ineye Komonibo', 'Brenda Barrientos', 'Sandy...",https://www.refinery29.com/en-us/2017/05/15511...,Is Selena Gomez's New Song About Justin Bieber...,1.494803e+09,http://www.refinery29.com,NaN,8882451342,fake,gossipcop,ccc8e821208ed614b4f2898262cb5328
736,https://variety.com/2018/music/news/scott-hutc...,A body found Thursday night is that of missing...,[],['Robert Mitchell'],https://variety.com/2018/music/news/scott-hutc...,"Scott Hutchison, Frightened Rabbit Frontman, F...",1.526042e+09,https://variety.com,NaN,934522,real,gossipcop,d2de9211a314a8c7ef2d36d1ee4fc764
16488,mashable.com/2016/07/19/lindsay-lohan-terroris...,Those who fail to learn from the history of Mi...,[],[],https://mashable.com/article/lindsay-lohan-ter...,Lindsay Lohan went full-on Mischa Barton with ...,1.468883e+09,http://mashable.com,NaN,3941312971,fake,gossipcop,38202acc6ffc3e0d86928909cf51abfa
20902,foxync.com/3539782/folks-lost-their-minds-over...,LIKE US ON FACEBOOK. FOLLOW US ON TWITTER AND ...,[],['Karen Clark'],https://foxync.com/3539782/folks-lost-their-mi...,After Giving Birth To Twins Beyonce Is Thicker...,1.502117e+09,http://foxync.com,NaN,5411760543,fake,gossipcop,7cf031894914956a4a03fd4513ab1977
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16335,www.newidea.com.au/meghan-markle-prince-harry-...,Sister-in-law Erin Specht has revealed the bom...,[],['New Idea'],https://www.newidea.com.au/meghan-markle-princ...,Meghan and Harry: We're having a baby already!,1.520194e+09,http://www.newidea.com.au,NaN,2338328497,fake,gossipcop,89082411c43679bca81e4f35060eed72
15721,https://variety.com/2018/tv/news/prince-harry-...,UPDATED: The eyes of the world turned to Winds...,[],['Robert Mitchell'],https://variety.com/2018/tv/news/prince-harry-...,Thousands Descend on Windsor for Wedding of Pr...,1.526713e+09,https://variety.com,NaN,934017,real,gossipcop,432c73ee288aaab2756b666169e4200e
9960,https://www.rollingstone.com/tv/tv-news/see-be...,"Ben Affleck, Matt Damon, J.J. Abrams and other...",[],['Elias Leight'],https://www.rollingstone.com/tv-movies/tv-movi...,"See Ben Affleck, Matt Damon Bring Jimmy Kimmel...",1.510670e+09,https://www.rollingstone.com,NaN,893697,real,gossipcop,5f8a9eb396b8775c506b951d8352f0b2
10905,https://www.countryliving.com/life/a22713929/j...,"Gender Reveal Baseballs 2 Pack, Pink and Blue ...",[],[],https://www.countryliving.com/life/a22713929/j...,Jason Aldean and Wife Brittany Kerr Revealed t...,1.534188e+09,https://www.countryliving.com,NaN,856338,real,gossipcop,fe66c57ec052e354c4dd48da5a2cbae1


In [6]:
df["title"] = df["title"].fillna("")
df["text"] = df["text"].fillna("")
df["article_md5"] = df.apply(lambda x: hashlib.md5(str(x["title"]+x["text"]).encode('utf-8')).hexdigest(), axis=1)

In [7]:
model = llama_chat_hf(size=MODEL_SIZE)

Loading checkpoint shards: 100%|██████████| 15/15 [04:14<00:00, 16.98s/it]


In [8]:
system_context = \
    """You are a helpful and unbiased news verification assistant. You will be provided with the title and the full body of text of a news article. Then, you will answer further questions related to the given article. Ensure that your answers are grounded in reality, truthful and reliable. {abstain_context}It is essential that you only answer objectively with one of the following options: {options}. Please do not answer with anything other than the option provided."""

prompt = """{title}\n{text}\n\n{question} ({options})"""
# abstain_context = "You are only allowed to answer with 'Unsure' if you do not have enough information or context to provide a reliable answer."
abstain_context = ""
# fake_news_context = "Fake news is false or inaccurate information, especially that which is deliberately intended to deceive." # Decreases performance.

In [9]:
def category_mapping(answer):
    if answer.lower().startswith("no"):
        category = 0
    elif answer.lower().startswith("yes"):
        category = 1
    else:
        category = -1

    return category

def load_cache(p):
    cache = []
    if os.path.exists(p):
        with open(p, "r") as f:
            for i, line in enumerate(f):
                cache.append(json.loads(line))
            
    return cache

def dump_cache(line, p):
    with open(p, "a") as f:
        f.write(json.dumps(line)+"\n")

In [ ]:
def process(verbose=False):
    preds = []
    trues = []
    processed_records = []
    with tqdm(total=len(df)*len(signal_df)) as pbar:
        for i, article_row in enumerate(df.itertuples()):
            if article_row.article_md5 in [row["article_md5"] for row in load_cache(CACHE_PATH)]:
                continue

            # ZS Question
            system_context_zs = system_context.format(options="Yes/No", abstain_context="")
            prompt_formatted = prompt.format(title=article_row.title, text=article_row.text, system_context=system_context_zs, question="Does this article contain misinformation?", options="Yes/No")
            try:
                answer_zs = model.prompt(prompt_formatted, allow_abstain=False)
            except torch.cuda.OutOfMemoryError as e:
                continue # stop processing this example
            
            category_zs = category_mapping(answer_zs)
            preds.append(category_zs)
            label_converter = lambda x: 0 if x == "real" else 1
            true = label_converter(article_row.objective)
            trues.append(true)

            acc = accuracy_score(trues, preds)
            f1 = f1_score(trues, preds, average="macro")
            num_yes = len([x for x in preds if x == 1])
            num_no = len([x for x in preds if x == 0])
            num_abstain = len([x for x in preds if x == -1])
            updated_description = f"Acc={acc*100:.2f}, F1={f1*100:.2f}, Total={i}, Num_Yes={num_yes}, Num_No={num_no}, Num_Abstain={num_abstain}"
            pbar.set_description(updated_description)
            
            if verbose:
                print(answer_zs)
                
                print(10*"-")
                print("Objective:", article_row.objective)

            processed = {}
            for j, question_row in enumerate(signal_df.itertuples()):
                system_context_ws = system_context.format(options="Yes/Unsure/No", abstain_context=abstain_context)
                prompt_formatted_ws = prompt.format(title=article_row.title, text=article_row.text, question=question_row.Question, options="Yes/Unsure/No")

                try:
                    answer_ws = model.prompt(prompt_formatted_ws, system_context=system_context_ws, allow_abstain=True)
                except torch.cuda.OutOfMemoryError as e:
                    break # stop processing this example and the next questions

                if verbose:
                    print(question_row.Question, category_ws)
                    print(answer_ws)
                    
                category_ws = category_mapping(answer_ws)
                processed[question_row._2] = category_ws
                pbar.update(1)
            
            processed["objective_pred"] = category_zs
            processed["objective_true"] = true
            processed["article_md5"] = article_row.article_md5
            dump_cache(processed, CACHE_PATH)
            pbar.update(1)

In [ ]:
process(verbose=VERBOSE)

In [24]:
preds = []
trues = []
processed_records = []
with tqdm(total=len(df)*len(signal_df)) as pbar:
    for i, article_row in enumerate(df.itertuples()):
        if article_row.article_md5 in [row["article_md5"] for row in load_cache(CACHE_PATH)]:
            continue

        # ZS Question
        system_context_zs = system_context.format(options="Yes/No", abstain_context="")
        prompt_formatted = prompt.format(title=article_row.title, text=article_row.text, system_context=system_context_zs, question="Does this article contain misinformation?", options="Yes/No")
        try:
            answer_zs = model.prompt(prompt_formatted, allow_abstain=False)
        except torch.cuda.OutOfMemoryError as e:
            continue # stop processing this example
        
        category_zs = category_mapping(answer_zs)
        preds.append(category_zs)
        label_converter = lambda x: 0 if x == "real" else 1
        true = label_converter(article_row.objective)
        trues.append(true)

        acc = accuracy_score(trues, preds)
        f1 = f1_score(trues, preds, average="macro")
        num_yes = len([x for x in preds if x == 1])
        num_no = len([x for x in preds if x == 0])
        num_abstain = len([x for x in preds if x == -1])
        updated_description = f"Acc={acc*100:.2f}, F1={f1*100:.2f}, Total={i}, Num_Yes={num_yes}, Num_No={num_no}, Num_Abstain={num_abstain}"
        pbar.set_description(updated_description)
        
        print(answer_zs)
        
        print(10*"-")
        print("Objective:", article_row.objective)

        processed = {}
        for j, question_row in enumerate(signal_df.itertuples()):
            system_context_ws = system_context.format(options="Yes/Unsure/No", abstain_context=abstain_context)
            prompt_formatted_ws = prompt.format(title=article_row.title, text=article_row.text, question=question_row.Question, options="Yes/Unsure/No")

            try:
                answer_ws = model.prompt(prompt_formatted_ws, system_context=system_context_ws, allow_abstain=True)
            except torch.cuda.OutOfMemoryError as e:
                break # stop processing this example and the next questions

            category_ws = category_mapping(answer_ws)
            print(question_row.Question, category_ws)
            print(answer_ws)
            processed[question_row._2] = category_ws
            pbar.update(1)
        
        processed["objective_pred"] = category_zs
        processed["objective_true"] = true
        processed["article_md5"] = article_row.article_md5
        dump_cache(processed, CACHE_PATH)
        pbar.update(1)

  0%|          | 0/9500 [00:00<?, ?it/s]

Acc=100.00, F1=100.00, Total=0, Num_Yes=0, Num_No=1, Num_Abstain=0:   0%|          | 0/9500 [00:01<?, ?it/s]

no
----------
Objective: real


Acc=100.00, F1=100.00, Total=0, Num_Yes=0, Num_No=1, Num_Abstain=0:   0%|          | 1/9500 [00:03<9:27:45,  3.59s/it]

Does the article fail to present any supporting evidence or arguments to substantiate its claims? 1
yes


...

no
----------
Objective: real


Acc=71.29, F1=60.32, Total=415, Num_Yes=32, Num_No=278, Num_Abstain=0:  65%|██████▍   | 6170/9500 [3:43:15<3:08:54,  3.40s/it]

Does the article fail to present any supporting evidence or arguments to substantiate its claims? 0
no


Acc=71.29, F1=60.32, Total=415, Num_Yes=32, Num_No=278, Num_Abstain=0:  65%|██████▍   | 6170/9500 [3:43:17<2:00:30,  2.17s/it]


KeyboardInterrupt: 